In [ ]:
import json
import numpy as np
import time
from PIL import Image
import math
import matplotlib.pyplot as plt
import random
import uuid
import pandas as pd

def f01(x,y,x0,y0,x1,y1,x2,y2):
    q = (x0*y1)-(x1*y0)
    u = y0-y1
    v = x1-x0
    
    return ((u*x) + (v*y) + q)/((u*x2) + (v*y2) + q)

def f12(x,y,x0,y0,x1,y1,x2,y2):
    q = (x1*y2)-(x2*y1)
    u = y1-y2
    v = x2-x1
    
    return ((u*x) + (v*y) + q)/((u*x0) + (v*y0) + q)

def f20(x,y,x0,y0,x1,y1,x2,y2):
    q = (x2*y0)-(x0*y2)
    u = y2-y0
    v = x0-x2
    return ((u*x) + (v*y) + q)/((u*x1) + (v*y1) + q)

# RayTracer
https://bytes.usc.edu/cs580/s22_CG-012-Ren/lectures/Lect_RT/GeomQueries/slides_intersections.html

In [ ]:
# Miscellaneous:
#  Determine image plane given camera @ (0,0,0) with a Field-of-View and how to 
#    convert the image plane into a pixelated image. 
#    Let Field-of-View = 120 deg (matching iPhone 13)
#  Create surface plane to show shadows from ray-tracing
#  Calculate sphere model (pending)
 

# Direction: P2-P1 (128, 100, -1) - (0, 0, 0) = (128, 100, -1)
# Normalize the above
class Ray:
    def __init__(self, p0, d, t = -1):
        # p0, origin of a Ray
        self.p0 = p0 
        self.d = d * (1 / np.linalg.norm(d)) 
        self.t = t
        # d, unit vector, direction of ray (normalized)
        # self.var = None
    
    def set_t(self, t):
        self.t = t
        
    def get_point(self):
        return np.add(self.p0, np.multiply(self.d, self.t))

class Plane:
    def __init__(self, p0, normal):
        self.p0 = p0 # point on the plane
        self.normal = normal
    
    def on_plane(self, p1):
        # Checks if point is on plane, not really useful but for reference
        return (np.dot(normal, np.subtract(self.p0-p1)) == 0)


# viewport of [1,1] and distance of 1 would make FOV 53 degrees
class RayTracer: 
    def __init__(self, scene, viewport, canvas):
        # Load scene data for rendering
        self.scene = scene
        # List in format of {"vw": vw, "vh": vh, "d": d}, where vw = viewport width, vh = viewport height, d = distance from viewport
        self.viewport = viewport
        # Canvas info in format of {"w": w, "h": h}
        self.canvas = canvas
    
    def canvasToViewport(self, cx, cy):
        # Convert a point on the canvas to viewport units
        vw = self.viewport.vw
        vh = self.viewport.vh
        cw = self.canvas.w
        ch = self.canvas.h

        vx = cx * vx/cw
        vy = cy * vh/ch
        vz = viewport.d

        return [vx, vy, vz]
        
    
    def get_intersect(self, ray, triangle_vertices, triangle_normal):
        # Calculate if a ray intersects with a plane that contains a triangle
        # Helps filter out unnecessary triangles and calculates important values
        #    such as normals, and points of intersection.
        # Returns t if it intersects and is in the triangle, -1 otherwise

        v0 = triangle_vertices["v0"]

        cam_ray = Ray(self.scene["camera"]["point"], self.scene["camera"]["direction"])
        plane = Plane(v0, triangle_normal)

        # t = (n⋅(p⋅s))/(n⋅d) where 
        # p = point on plane
        # s = ray start point
        # d = camera direction
        # n = plane normal

        denominator = np.dot(n, d)

        # Checks if camera ray is parallel to the plane, in which case the ray will never hit the plane
        if denominator < 0.00001:
            return -1

        # Calculate t value, then plug into formula to get the point on the plane
        t = np.dot(plane.normal, np.dot(plane.p0, cam_ray.p0)) / denominator 

        plane.set_t(t)
        point_on_plane = plane.get_point()
        
        # Checks to see if the point is on the plane
        if self.inTriangle(triangle_vertices, plane.normal, point_on_plane):
            return point_on_plane

        return -1
    
    # def inTriangle(self):
    #     # Calculate if a ray intersect falls within a triangle
    #     # Project vertices of triangle onto 2D plane then you can 
    #     # use barycentric coordinates like in the homework

    def inTriangle(self, triangle_vertices, normal, point_on_plane):

        # Check if in triangle not using barycentric coordinates (barycentric is better)

        v0 = triangle_vertices["v0"]
        v1 = triangle_vertices["v1"]
        v2 = triangle_vertices["v2"]

        # Calculate the edge vectors
        v0v1_edge = np.subtract(v1, v0)
        v1v2_edge = np.subtract(v2, v1)
        v2v0_edge = np.subtract(v0, v2)

        # Get vectors from vert to point on plane
        v0_point = np.subtract(point_on_plane, v0)
        v1_point = np.subtract(point_on_plane, v1)
        v2_point = np.subtract(point_on_plane, v2)
        
        # Get cross product and compare it to normal
        v0_cross = np.cross(v0v1_edge, v0_point)
        v1_cross = np.cross(v1v2_edge, v1_point)
        v2_cross = np.cross(v2v0_edge, v2_point)
        
        # If the dot == 0, normal and cross product are opposite to each other, so it will not be in the plane
        v0_normal_dot = np.dot(v0_cross, normal) > 0
        v1_normal_dot = np.dot(v1_cross, normal) > 0
        v2_normal_dot = np.dot(v2_cross, normal) > 0

        return v0_normal_dot and v1_normal_dot and v2_normal_dot
    
    def castShadowRay(self):
        # Attempt to trace shadow ray back to light source 
        # If hits another poylgon, color should be a shadow
        # If shadow ray can be traced back to light source, calculate lighting to fill pixel
        
        
        return

In [ ]:
# Lighting Model to get ADS 
class Lighting:
    def __init__(self,
                 camera,
                 light
                ):
        self.L = self.normalize(light[1]['from']) # TODO: include from-to explicitly incase data is different in future
        self.V = self.normalize(camera['from'])
        
        # Ambient lighting parameters
        self.aColor = np.array(light[0]['color'])
        self.Ia = light[0]['intensity']
        
        # Directional lighting parameters
        self.dColor = np.array(light[1]['color']) #Directional Color
        self.Ie = light[1]['intensity']
        
    def calculateADS(self, n, material):
        n = self.normalize(n)
        # Cases that come up in shading...
        if np.dot(n, self.L) < 0 and np.dot(n, self.V) < 0: n*=-1 # CASE 1: 
        elif np.dot(n, self.L) * np.dot(n, self.V) < 0: emissive = np.array([0,0,0])
        
        Ka, Kd, Ks = material['Ka'], material['Kd'], material['Ks']
        S = material['n']
        
        a = self.ambient(Ka)
        d = self.diffuse(Kd, n)
        s = self.specular(Ks, n, S)
        
        return a+d+s
            
    def ambient(self, Ka):
        return Ka * self.aColor * self.Ia
    
    def diffuse(self, Kd, n):
        dotnl = np.dot(n, self.L)
        dotnl = np.clip(dotnl, 0, 1)
        return Kd * self.dColor * dotnl * self.Ie
    
    def specular(self, Ks, n, S):
        r = self.reflectionVector(n)
        
        dotre = np.dot(r, self.V)
        dotre = np.clip(dotre, 0, 1) # Clamp RE
        dotre = dotre**S # Apply Shininess
        return Ks * self.dColor * dotre * self.Ie
    
    def reflectionVector(self, n):
        # Calculate Normalized Reflection Vector
        r = 2*(np.dot(n, self.L))*n - self.L
        return self.normalize(r)
        
    def normalize(self, pt): 
        l = math.sqrt(pt[0]**2 + pt[1]**2 + pt[2]**2)
        return np.array([pt[0]/l, pt[1]/l, pt[2]/l])

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b1d9b2dc-0aa6-4b59-8bd7-0b3848a2d4a6' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>